# TPM034A Machine Learning for socio-technical systems 
## `Lab session 04:  Explainable AI and energy prediction`

**Delft University of Technology**<br>
**Q2 2024**<br>
**Instructor:** Giacomo Marangoni <br>
**TAs:**  Francisco Garrido Valenzuela & Lucas Spierenburg <br>

### `Instructions`

**Lab sessions aim to:**<br>
* Show and reinforce how models and ideas presented in class are used in practice.<br>
* Help you gather hands-on machine learning skills.<br>

**Lab sessions are:**<br>
* Learning environments where you work with Jupyter notebooks and where you can get support from TAs and fellow students.<br> 
* Not graded and do not have to be submitted. 
### `Use of AI tools`
AI tools, such as ChatGPT and Co-pilot, are great tools to assist with programming. Moreover, in your later careers you will work in a world where such tools are widely available. As such, we **encourage** you to use AI tools **effectively** (both in the lab sessions and assignments). However, be careful not to overestimate the capacity of AI tools! AI tools cannot replace you: you still have to conceptualise the problem, dissect it and structure it, to conduct proper analysis and modelling. We recommend being especially **reticent** with using AI tools for the more conceptual and reflective oriented questions. 
### `Google Colab workspace set-up`

Uncomment the following cells code lines if you are running this notebook on Colab

In [ ]:
#!git clone https://github.com/TPM034A/Q2_2024
#!pip install -r Q2_2024/requirements_colab.txt
#!mv "/content/Q2_2024/Lab_sessions/lab_session_04/data" /content/data

### `Application: Explaining the prediction of the next 24-hour electricity load` <br>

#### **Introduction**
In this lab session we will train a Linear regression (LR) and a Random forest regressor (RF) to predict the **next 24-hour electricity load** of a household. We will then use the tools of Explainable AI, in particular PDP, LIME and SHAP, to better understand how specific prediction are influenced by different features.

#### **Data**

1. `data/load.pkl`: A pickle file with a pandas.DataFrame of overall Wh hourly energy consumption of a household from the REFIT Electrical Load Measurements dataset, collected over a period of about two years.
2. `data/devices.pkl`: A pickle file with a pandas.DataFrame of normalized weather variables: `dwpt` is Dew Point (related to moisture), `rhum` is relative humidity, `temp` is temperature, `wdir` is wind direction, `wspd` is wind speed.


### Preparation

In [ ]:
import pandas as pd

if False:
    load_df_orig = pd.read_pickle('original_data/load_df.pkl')
    load_df_orig.head()
    
    load_df['load'].to_pickle('data/load.pkl')
    
    load_df[['temp', 'dwpt', 'rhum', 'wdir', 'wspd']].to_pickle('data/weather.pkl')
    
    load_df = pd.read_pickle('original_data/load_df.pkl')
    load_df.head()
    
    load_df['Aggregate'].rename('load').to_pickle('data/load.pkl')
    
    prices_df = pd.read_pickle('original_data/price_df.pkl')
    prices_df.rename('price').to_pickle('../../Assignments/assignment_04/data/price.pkl')
    
    load_df = pd.read_pickle('original_data/load_df.pkl')
    load_df.columns
    
    devices = ['Toaster', 'Fridge-Freezer', 'Freezer',
           'Tumble Dryer', 'Dishwasher', 'Washing Machine', 'Television',
           'Microwave', 'Kettle']
    #load_df[devices] = (load_df[devices] > 1).astype(int)
    load_df['Load'] = load_df['Aggregate']
    load_df.head()
    load_df['Load'].to_pickle('../../Assignments/assignment_04/data/load.pkl')
    load_df[devices].to_pickle('../../Assignments/assignment_04/data/devices.pkl')
    load_df[['dwpt', 	'rhum', 	'temp', 	'wdir', 	'wspd']].to_pickle('../../Assignments/assignment_04/data/weather.pkl')
    
    load_df[['Aggregate', 'Toaster', 'Fridge-Freezer', 'Freezer',
           'Tumble Dryer', 'Dishwasher', 'Washing Machine', 'Television',
           'Microwave', 'Kettle']].to_pickle('../../Assignments/assignment_04/data/devices.pkl')

### Load and preprocess the datasets

In [ ]:
# Use pd.read_pickle to load 'data/load.pkl'

# Check that you get a similar pd.Series:
# Time
# 2013-09-25 19:00:00    410.766578
# 2013-09-25 20:00:00    417.421053
# 2013-09-25 21:00:00    508.165821
# ...

In [ ]:
import pandas as pd
load = pd.read_pickle('data/load.pkl')
load.head()

In [ ]:
# How many observations does the dataset have? What is the overall time range?

In [ ]:
load.sort_index(inplace=True)
load.index[[0,-1]]
# or
load.index.to_series().describe()

In [ ]:
# Use pd.read_pickle to load 'data/weather.pkl'

# Check that you have a pd.DataFrame with columns
# ['temp', 'dwpt', 'rhum', 'wdir', 'wspd']
# and index equal to the load index

In [ ]:
weather = pd.read_pickle('data/weather.pkl')
weather.head()

In [ ]:
# Add an "hour" column to the load df, corresponding to the integer hour of the index
# e.g. 2013-09-25 19:00:00 --> 19
# Hint: convert index into a pd.Series and use the .hour accessor

In [ ]:
load_df = load.to_frame()
load_df['hour'] = load_df.index.hour
load_df.head()

In [ ]:
# Add 3 columns to the load df
# load_lag_24, load_lag_48 and load_lag_72
# corresponding to the value of load 24, 48 and 72 hourse before, respectively
# Hint: use the .shift function

# check as example the first five 21:00 samples to make sure you computed the lags correctly

In [ ]:
for x in [24, 48, 72]:
    load_df[f'load_lag_{x}'] = load_df['load'].shift(x)
load_df[load_df.hour == 21].head(5)

In [ ]:
# Add 7 dummy columns, each one with value 0 or 1 for each day of the week of the corresponding index,
# named "day_name_monday", "day_name_tuesday", ... , "day_name_saturday" (skipping Sunday)
# Hint: use index.dayofweek and pd.get_dummies

# check as example the first seven 21:00 samples to make sure you computed the day_name columns correctly

In [ ]:
import calendar
day_name_list = [x.lower() for x in calendar.day_name]
load_df2 = load_df.copy()
load_df2['day_name'] = load_df2.index.dayofweek
load_df2['day_name'] = load_df2['day_name'].replace(dict(zip(range(7), day_name_list)))
load_df2 = pd.get_dummies(
    load_df2,
    prefix='day_name',
    columns=['day_name'],
    dtype=int
).drop('day_name_sunday', axis=1)
load_df2[load_df2.hour == 21].head(7)

In [ ]:
# Merge the weather and the load df
# Hint: use df.join

In [ ]:
load_df3 = load_df2.join(weather)
load_df3.head()

In [ ]:
# Plot the load of a single day, e.g. 2013-12-12
# Does the shape make sense?

In [ ]:
# Peaks possibly at breakfast, pre and post dinner
load_df3.loc['2013-12-12','load'].plot()

In [ ]:
# Plot the average and confidence interval of hourly load for weekdays vs weekends from '2013-11-01' onwards
# Hint: use seaborn.lineplot
# Do you observe any difference?

In [ ]:
# During weekdays higher morning / evening peaks, possibly following classic workday routine.
import seaborn as sb
load_df3b = load_df3.copy()
load_df3b['weekday'] = (load_df3b.index.dayofweek <= 4)
sb.lineplot(x='hour', y='load', hue='weekday', errorbar=('ci', 90), n_boot=200, data=load_df3b.loc['2013-11-01':])

In [ ]:
# Add dummies "hour_1", "hour_2", ... "hour_23" (skipping "hour_0"), equal to 1 for the corresponding index hour
# Hint: use pd.get_dummies

In [ ]:
load_df4 = pd.get_dummies(
    load_df3,
    prefix='hour',
    columns=['hour'],
    dtype=int
).drop('hour_0', axis=1)

### Train a linear regression model

In [ ]:
# Train a linear regression model to predict load of 2014-12-12 based on features computed above.
# Use the period 2013-11-01 to 2014-12-11 for training.

In [ ]:
load_df5 = load_df4.dropna()
X_train_lr = load_df5.loc['2013-11-01':'2014-12-11', load_df5.columns != 'load']
y_train_lr = load_df5.loc['2013-11-01':'2014-12-11', 'load']
X_test_lr = load_df5.loc['2014-12-12', load_df5.columns != 'load']
y_test_lr = load_df5.loc['2014-12-12', 'load']
X_train_lr.head()

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
model_lr = LinearRegression().fit(X_train_lr, y_train_lr)

In [ ]:
# Compute the root mean squared error

In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
y_pred_lr = model_lr.predict(X_test_lr)
mse_lr = mean_squared_error(y_test_lr, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
print(rmse_lr)

### Explain a LR model with LIME

In [ ]:
# Compute the indices of the variables that are categorical

In [ ]:
categorical_prefixes = ["day_name", "hour_"]
categorical_indices_lr = [
    i for i, key in enumerate(X_train_lr.columns) 
    if any(key.startswith(prefix) for prefix in categorical_prefixes)
]
print(categorical_indices_lr)

In [ ]:
# Provide a LIME explanation of the prediction for day 2013-12-05 at 12:00.
# Consider the train dataset.
# What features drive the explanation?
# Hint: Use LimeTabularExplainer from lime.lime_tabular

In [ ]:
from lime import lime_tabular

lime_explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_train_lr.values,
    feature_names=X_train_lr.columns,
    categorical_features=categorical_indices_lr,
    mode='regression'
)

instance_to_explain = X_train_lr.loc['2013-12-05 12:00'].values
explanation = lime_explainer.explain_instance(
    data_row=instance_to_explain,
    predict_fn=model_lr.predict,
)

explanation.show_in_notebook()

In [ ]:
# Compare the explanation with the model coefficients

In [ ]:
(pd.Series(model_lr.coef_, index=X_train_lr.columns)
 .abs()
 .sort_values(ascending=False)
 .head())

In [ ]:
# What was the actual consumption value for that hour?

In [ ]:
y_train_lr.loc['2013-12-05 12:00']

### Explain a LR model with SHAP

In [ ]:
# Compute SHAP values for the train dataset.
# Hint: use shap.LinearExplainer and shap.sample as background data matrix

In [ ]:
import shap
X100_lr = shap.sample(X_train_lr, 1000)
explainer_lr = shap.LinearExplainer(model_lr, X100_lr)
shap_values_lr = explainer_lr(X_train_lr)

In [ ]:
# Explain the prediction for day 2013-12-05 at 12:00

In [ ]:
shap_values_lr.shape

In [ ]:
idx2explain_lr = y_train_lr.index.get_loc('2013-12-05 12:00')
idx2explain_lr

In [ ]:
shap.plots.waterfall(shap_values_lr[idx2explain_lr])

In [ ]:
# Agreement on + impact of hour12 and to a less extent - impact of non-peak hours
# Disagreement on importance of lags
# Difference in how background data distribution and use or not of perturbed local model

### Train a Random Forest Regressor

In [ ]:
# Train a random forest regressor model to predict load of 2014-12-12.
# Use the period 2013-11-01 to 2014-12-11 for training.
# Do not use the dummies computed above. Use the following features instead:
# ['hour', 'load_lag_24', 'load_lag_48', 'load_lag_72', 'temp',
#    'dwpt', 'rhum', 'wdir', 'wspd', 'weekday']
# where hour and weekday are int

In [ ]:
from sklearn.ensemble import RandomForestRegressor
load_df6 = load_df.join(weather).dropna()
load_df6['weekday'] = load_df6.index.dayofweek.astype(int)
load_df6['hour'] = load_df6['hour'].astype(int)
load_df6.head()

In [ ]:
X_train_rf = load_df6.loc['2013-11-01':'2014-12-11', load_df6.columns != 'load']
y_train_rf = load_df6.loc['2013-11-01':'2014-12-11', 'load']
X_test_rf = load_df6.loc['2014-12-12', load_df6.columns != 'load']
y_test_rf = load_df6.loc['2014-12-12', 'load']
model_rf = RandomForestRegressor(random_state=0).fit(X_train_rf, y_train_rf)

In [ ]:
# What is the root mean squared error for the test set? Does the accuracy improve with respect to the linear model?

In [ ]:
# RF improves on the LR
y_pred_rf = model_rf.predict(X_test_rf)
mse_rf = mean_squared_error(y_test_rf, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
print(rmse_rf)

### Explain a RF model with LIME

In [ ]:
categorical_indices_rf = [
    i for i, key in enumerate(X_train_rf.columns) 
    if key in ['hour', 'weekday']
]
print(categorical_indices_rf)

In [ ]:
# Provide a LIME explanation of the prediction for test day 2014-12-12 at 20:00.
# What features drive the explanation?
# Hint: Use LimeTabularExplainer from lime.lime_tabular

In [ ]:
from lime import lime_tabular

lime_explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_train_rf.values,
    feature_names=X_train_rf.columns,
    categorical_features=categorical_indices_rf,
    mode='regression'
)

instance_to_explain = X_test_rf.loc['2014-12-12 20:00'].values
explanation = lime_explainer.explain_instance(
    data_row=instance_to_explain,
    predict_fn=model_rf.predict,
)

explanation.show_in_notebook()

### Explain a RF model with PDP

In [ ]:
# What is the expected energy consumption knowing the weekday?
# What is the weekday with the highest/lowest expected consumption?
# 
# Hint: use shap.partial_dependence_plot

In [ ]:
fig, ax = shap.partial_dependence_plot(
    "weekday",
    model_rf.predict,
    X_train_rf,
    model_expected_value=True,
    feature_expected_value=True,
    show=False,
    ice=False,
)

In [ ]:
# Compare with the mean load of the train set grouped by weekday: are they different? If so, why?

In [ ]:
# The numbers are aligned but different. 
# The PDP considers the individual impact of weekday all other features being equal,
# and reflects the importance that the model attributes to weekday.
# The groupby mean is a descriptive statistics of the training data.
y_train_rf.groupby(y_train_rf.index.weekday).mean()

In [ ]:
# What is the expected energy consumption knowing the hour? What about temperature? Does the model behavior make sense?

In [ ]:
# Hour: classic double peak, likely at breakfast and dinner
fig, ax = shap.partial_dependence_plot(
    "hour",
    model_rf.predict,
    X_train_rf,
    model_expected_value=True,
    feature_expected_value=True,
    show=False,
    ice=False,
)

In [ ]:
# Temperature: classic U-shape, people more likely to stay home for low/high temps
fig, ax = shap.partial_dependence_plot(
    "temp",
    model_rf.predict,
    X_train_rf,
    model_expected_value=True,
    feature_expected_value=True,
    show=False,
    ice=False,
)

### Explain a RF model with SHAP

In [ ]:
# Plot the SHAP values of each feature for each of the TEST samples.
# Hint: use shap.TreeExplainer and shap.plots.beeswarm

In [ ]:
X100 = shap.sample(X_train_rf, 100, random_state=0)
explainer_rf = shap.TreeExplainer(model_rf, X100)
shap_values_rf = explainer_rf(X_test_rf)

In [ ]:
shap.plots.beeswarm(shap_values_rf)

In [ ]:
# What is the most predictive feature? What is the least predictive? How do you interpret it?
# Hint: use shap.plots.bar

In [ ]:
# Most predictive:
shap.plots.bar(shap_values_rf)

In [ ]:
# Explain the 15:00 and 20:00 prediction of the test. What observations can you make? How does the 20:00 explanation compare to LIME one above?
# Hint: use shap.plots.waterfall

In [ ]:
shap.plots.waterfall(shap_values_rf[15])

In [ ]:
# In general, high autocorrelation (<->lag variables) and habit-driven (<->hour) consumption.
# In the evening, less influence of weather variables and weekday.
# Compared to LIME explanation above, hour and lags do matter more than other features in both tools, but less clear distinction of lags vs hours using local model.
shap.plots.waterfall(shap_values_rf[20])

### Reflections

In [ ]:
# Reflect on how the information above could be useful for environmental reasons.

In [ ]:
# For the energy provider, better planning to cover peaks, and info to best design energy demand response.
# For the consumer, better awareness of patterns of consumption, and incentive to shift load to align with renewables availability.